<a href="https://colab.research.google.com/github/runtheorun-exe/ir-colab/blob/sbert/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing and downloading all dependencies

In [ ]:
%tensorflow_version 1.x
!pip install bert-serving-client
!pip install -U bert-serving-server[http]
!pip install -U sentence_transformers
!pip install elasticsearch
!pip install flask-ngrok
!pip install flask==0.12.2 

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
from bert_serving.client import BertClient
import json
import requests
from elasticsearch import Elasticsearch


In [ ]:
!unzip uncased_L-12_H-768_A-12.zip
!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 out.file2>&1&

Ensure bert was installed succesfully


Search with SentenceTransformers


In [ ]:
df = pd.read_excel (r'/content/drive/MyDrive/cordis-h2020projects.xlsx')
objective_list=df['objective'].values.tolist()
title_list=df['title'].values.tolist()
id_list=df['id'].values.tolist() 
corpus_embeddings = model.encode(objective_list, convert_to_tensor=True)
user_query="REPAC"#input("Enter search query:")

In [ ]:
top_k = min(20, len(objective_list))
query_embedding = model.encode(user_query, convert_to_tensor=True)
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

print("\n\n======================\n\n")
print("Query:", user_query)
print("\nTop 20 results:")
flag=0
for score, idx in zip(top_results[0], top_results[1]):
    flag+=1
    print(flag,":",title_list[idx],"\nProject ID:",id_list[idx],"\nObjective:",objective_list[idx], "(Score: {:.4f})".format(score),"\n\n")





Query: REPAC

Top 20 results:
1 : Repetition, Parallelism and Creativity: An Inquiry into the Construction of Meaning in Ancient Mesopotamian Literature and Erudition 
Project ID: 803060 
Objective: REPAC is a contribution to the intellectual history of Ancient Mesopotamia, one of mankind’s earliest civilizations. The project investigates the relationship between textual form and function in Mesopotamian literature and scholarly compositions. It focuses on exact and variant repetition as a text structuring device and demonstrates that the use of repetition is grounded in a fundamental aspect of the worldview of Mesopotamian learned scribes. REPAC studies literary, magical and mantic texts written on clay tablets inscribed in the Cuneiform script in the Akkadian language dating from the first millennium BCE. Parallelism, refrain, ring composition and similar types of regular, part-repetitive patterning occur on all levels of these erudite textual genres. Textual elements which are s

Prepare Elasticsearch (creating indices and filling them)


In [ ]:
bc=BertClient()

es = Elasticsearch(
    cloud_id="ir-bert:ZXVyb3BlLXdlc3QzLmdjcC5jbG91ZC5lcy5pbyRjMGVlMTFkMmMyMDA0NTNiOTM4MmQ2ZWU2Y2EzM2JlOSQ0NzkyMzM2N2UwMTE0MDQ0OTRkNGI2ODBiYTdkMjI5NA==",
    http_auth=("elastic", "lMukP77h1QGrhqjSXQrK9Ans")
)
try:
  request_body = {
          "settings" : {
              "number_of_shards": 1,
              "number_of_replicas": 1
          },

          "mappings": {
              "properties": {
                  "title": {"type": "text"},
                  "objective":{"type": "text"},
                  "id": {"type": "text"},
                  "body_vector": {"type":  "dense_vector", "dims":  768}
              }
          }
      }
  es.indices.create(index = 'cordis2020', body = request_body)
except:
  pass

flg = 0 # used to iterate through the premade lists
lng = len(id_list)
for x in range(0, 50): #for the sake of expedience, during development the index range was limited to less than a tenth of the original size
  project={
      'title': title_list[flg],
      'objective': objective_list[flg],
      'body_vector': model.encode([corpus_embeddings])[0].tolist(),
      'id': id_list[flg]}
  es.index(index='cordis2020', id=flg, body=project)
  print(flg)
  flg+=1

print('Process complete')


In [ ]:
SEARCH_SIZE = 20 # we need 20 hits, thus search_size is set to 20

script_query = {
          "script_score": {
              "query": {"match_all": {}},
              "script": {
                  "source": "cosineSimilarity(params.query_vector, 'body_vector') + 1.0",
                  "params": {"query_vector": query_embedding}
              }
          }
      }

res = es.search(
  index='cordis2020',  # name of the index
  body={
      "size": SEARCH_SIZE,
      "query": user_query,
      "_source": {"includes": ["title", "objective", "body_vector", "id"]}
      }
  )

results = []
for hit in res['hits']['hits']:
  print("help")
  results.append(base_element.format(url=hit['_source']['link'], title=hit['_source']['title']))

Beyond this point, the project may or may not be functional. The HTML interface may or may not be complete.

In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()  